# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json



#    Need for slowing down the query loop and date stamps
import time

# Import API key
from api_keys import g_key

In [2]:
class GooglePlaces(object):
    def __init__(self, apiKey):
        super(GooglePlaces, self).__init__()
        self.apiKey = apiKey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#weather_data_df = pd.read_csv("big_weather_data_df.csv")

#works
#weather_data_o112 = pd.read_csv("../output_data/com_weather_data_df.csv")
#weather_data_df = weather_data_o112
#weather_data_o112
weather_data_df = pd.read_csv("output_2/big_weather_data_df.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
 # Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

#extra import probably delete
# Fill NaN values and convert to float
humid = weather_data_df["Humidity (RH)%"].astype(float)

In [5]:
gmaps.configure(api_key = g_key)

In [6]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0],zoom_level = 2)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid , 
                                 dissipating=False, max_intensity=85,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

#label map


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * Drop any rows will null values.

    * A max temperature lower than 80 degrees but higher than 70.
    * Wind speed less than 10 mph.
    * Zero cloudiness.
    * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [7]:
#weather_data_df

In [8]:
mod_temp_df = weather_data_df.loc[(weather_data_df["Temp F"] < 80) & (weather_data_df["Temp F"] > 70) & 
                                  (weather_data_df["wind speed (mph)"] < 6) &
                                  (weather_data_df["Cloud% Cover"] == 0)].copy()

In [9]:
mod_temp_df

,Unnamed: 0,City ID,City,Country Code,Latitude,Longitude,Temp Kelvin,Temp C,Temp F,Humidity (RH)%,Cloud% Cover,wind speed (m/s),wind speed (mph),Time stamp
190,190,315468,Ergani,TR,38.27,39.76,299.10,26.0,79.0,36,0,2.10,4.67,['21 Oct 2020 09:01:15']
309,309,2094027,Kieta,PG,-6.22,155.63,299.44,26.0,79.0,77,0,1.46,3.24,['21 Oct 2020 09:01:15']
374,374,1062553,Mahanoro,MG,-19.90,48.80,297.58,24.0,76.0,64,0,2.34,5.20,['21 Oct 2020 09:01:15']


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#hotel_df

Basic

The Basic category includes the following fields:
business_status, formatted_address, geometry, icon,name, permanently_closed (deprecated), photos, place_id, plus_code, types

Required parameters

key — Your application's API key. This key identifies your application. 

location — The latitude/longitude around which to retrieve place information. This must be specified as latitude,longitude.

radius — Defines the distance (in meters) within which to return place results. The maximum allowed radius is 50 000 meters. Note that radius must not be included if rankby=distance (described under Optional parameters below) is specified.
Optional parameters

keyword — A term to be matched against all content that Google has indexed for this place, including but not limited to name, type, and address, as well as customer reviews and other third-party content.

type — Restricts the results to places matching the specified type. Only one type may be specified (if more than one type is provided, all types following the first entry are ignored). See the list of supported types.

https://developers.google.com/places/web-service/search


In [11]:
Hotel_Name = []
hotel_address = []
hotel_rating = []

hotel_lat = []
hotel_lon = []




# params dictionary to update each iteration
params = {
            "radius": 50000,
            "types": "lodging",
            "keyword": "hotel",
            "key": g_key
}

# Use the lat/lng we recovered to identify airports

for index, row in mod_temp_df.iterrows():
    

    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    
#make request and print url

    name_address = requests.get(base_url, params=params)
    #print(name_address)

########     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)
 #########   # convert to json
    name_address = name_address.json()
#    print(json.dumps(name_address, indent=4, sort_keys=True))

#name_address

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    
    
    try:
####################   Stratagy 1 gets a new datafame based on hotel. Prefered method
###################    Works well but can't find a way to get country and city data from json
####################   Cant revese look up but working on it

        #list stratagy for new data frame does not have country
        Hotel_Name.append(name_address["results"][0]["name"])
        hotel_address.append(name_address["results"][0]["vicinity"])
        hotel_rating.append(name_address["results"][0]["rating"])
        
        hotel_lat.append(name_address["results"][0]["geometry"]["location"]["lat" ])
        hotel_lon.append(name_address["results"][0]["geometry"]["location"]["lng"])
        
        
###############       Stratagy two: 
################   adds hotel name and street addres data, json does not have country or city
################   data for the hotel. This dataframe will have the city from the orginal
################   search but not the hotel city data. Needs a reverse look up.


        #mod_temp_df.loc[index,"Hotel Name"] = name_address["results"][0]["name"]
        #mod_temp_df.loc[index,"Hotel Address"] = name_address["results"][0]["vicinity"]
        #mod_temp_df.loc[index,"Hotel Rating"] = name_address["results"][0]["rating"]
        
        #mod_temp_df.loc[index,"Lat"] = name_address["results"][0]["geometry"]["location"]["lat" ]
        #mod_temp_df.loc[index,"Lon"] = name_address["results"][0]["geometry"]["location"]["lng"]


    
    except (KeyError, IndexError):
        print("Hotel not found")


In [12]:
#chec hotel name list probably delete
#print(Hotel_Name)

In [13]:
# zips hotel dataframe
hotel_df = pd.DataFrame(list(zip(Hotel_Name, hotel_address, hotel_rating, hotel_lat, hotel_lon)), 
                    columns =["Hotel Name", "Hotel Address", "Hotel Rating", "Lat", "Lng"])


In [14]:
hotel_df 

,Hotel Name,Hotel Address,Hotel Rating,Lat,Lng
0,The Green Park Hotel Diyarbakır *****,"No:, Gazi Cd. No:21, Sur",4.6,37.910188,40.236627
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832


In [15]:
mod_temp_df

,Unnamed: 0,City ID,City,Country Code,Latitude,Longitude,Temp Kelvin,Temp C,Temp F,Humidity (RH)%,Cloud% Cover,wind speed (m/s),wind speed (mph),Time stamp
190,190,315468,Ergani,TR,38.27,39.76,299.10,26.0,79.0,36,0,2.10,4.67,['21 Oct 2020 09:01:15']
309,309,2094027,Kieta,PG,-6.22,155.63,299.44,26.0,79.0,77,0,1.46,3.24,['21 Oct 2020 09:01:15']
374,374,1062553,Mahanoro,MG,-19.90,48.80,297.58,24.0,76.0,64,0,2.34,5.20,['21 Oct 2020 09:01:15']


In [16]:
hotel_df 

,Hotel Name,Hotel Address,Hotel Rating,Lat,Lng
0,The Green Park Hotel Diyarbakır *****,"No:, Gazi Cd. No:21, Sur",4.6,37.910188,40.236627
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832


In [17]:
hotel_df["Match Address"] = ""
hotel_df 

,Hotel Name,Hotel Address,Hotel Rating,Lat,Lng,Match Address
0,The Green Park Hotel Diyarbakır *****,"No:, Gazi Cd. No:21, Sur",4.6,37.910188,40.236627,
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209,
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832,


In [29]:
# Use for revese look up recycled from preivous result. need difrent base urls

# Hotel variable
hotel_addresF = []
hotel_addresCC =[]

# Loop through narrowed down dataframe to get nearest hotel
#for hotel in range(len(hotel_df["Hotel Name"])):

#for hotel in hotel_df.iterrows():
    
    #lat = hotel_df.loc[city]["Lat"]
    #lng = hotel_df.loc[city]["Lng"]
    
    #hotel_coords = f"{lat},{lng}"
    
    #params = {
        #"location": hotel_coords, 
        #"types": "lodging",
        #"radius": 1,
        #"key": g_key
    #}
    #####################################################
    #geocode url
    #geo_base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    #params_Q = {
        #"location": hotel_coords_hardcode,
        #"types": hotel,
        #"radius": 1,
        #"key": g_key
    #}
    
    #this url is same as above. JSON will not give city and country data
Nearby_base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
# Text search
#text_base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"


# Text search
#    text_base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

#test_hotel_loop = hotel_df["Hotel Address"]
    
# Text search
#params = {
##"query": "test_hotel_loop",
#    key": g_key
        
#    }
    

x = "37.921952,40.147579"


    
for hotel in hotel_df["Hotel Address"]:
     
    params_Q = {
        "location": x,
        "types": hotel,
        "radius": 100,
        "key": g_key
    }
    
   
    # Text search
    text_base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    
    text_base_2_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    
    response = requests.get(text_base_url, params = params_Q)
    
    #print(response)
    
    
    hotel_location = response.json()
    
        #print(hotel_location)
    #print(json.dumps(hotel_location, indent=4, sort_keys=True))
    try:
        #hotel_df.loc[index,"Match Address"] = name_address["results"][0]["formatted_address"]
        #hotel_df.loc[index, "hotel_Lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        hotel_addresF.append(hotel_location["results"][0]["formatted_address"])
        hotel_addresCC.append(hotel_location["results"][0]["plus_code"]["compound_code"])
        
    except:
        pass 

print(hotel_addresCC)
print("")
#print(test_hotel_loop)

[]



In [30]:
hotel_addresCC

[]

In [31]:
hotel_df

,Hotel Name,Hotel Address,Hotel Rating,Lat,Lng,Match Address
0,The Green Park Hotel Diyarbakır *****,"No:, Gazi Cd. No:21, Sur",4.6,37.910188,40.236627,
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209,
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832,


In [32]:
twar =[]

for hotel in hotel_df["Hotel Address"]:
    twar = hotel_df["Hotel Address"]
print(twar)
    
    

0      No:, Gazi Cd. No:21, Sur
1    Section 16, Lot 36,, Arawa
2               Ambalamangahazo
Name: Hotel Address, dtype: object


In [33]:
########     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)
 #########   # convert to json
    name_address = name_address.json()
#    print(json.dumps(name_address, indent=4, sort_keys=True))

UnboundLocalError: local variable 'name_address' referenced before assignment

In [34]:
hotel_df 

,Hotel Name,Hotel Address,Hotel Rating,Lat,Lng,Match Address
0,The Green Park Hotel Diyarbakır *****,"No:, Gazi Cd. No:21, Sur",4.6,37.910188,40.236627,
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209,
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832,


In [35]:
H_lat=[]
H_lon=[]
cord = []


for hotel in hotel_df["Hotel Address"]:
    H_lat = hotel_df["Lat"]
    H_lon = hotel_df["Lng"]
    print(f"{hotel_df["Lat"]},{hotel_df["Lng"]}")


SyntaxError: invalid syntax (<ipython-input-35-953b9a528ecd>, line 9)

In [53]:
for hotel in hotel_df["Hotel Address"]:
    lat = hotel_df.loc[city]["Lat"]
    lng = hotel_df.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

NameError: name 'city' is not defined

In [49]:
x = "-6.233893,155.566209"

#lat = hotel_df.loc[city]["Lat"]
#lng = hotel_df.loc[city]["Lng"]
#lat = hotel_df["LAt"]
#lat = hotel_df["Lng"]


#for hotel in hotel_df["Hotel Address"]:

params_Q = {
        "location": x,
        "types": hotel,
        "radius": 100,
        "key": g_key
}


# Text search
text_base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

text_base_2_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

response = requests.get(text_base_url, params = params_Q)

#print(response)


hotel_location = response.json()

#print(json.dumps(hotel_location, indent=4, sort_keys=True))

#print(hotel_location["results"][0]["name"])

print(hotel_location["results"][0]["name"])
print(hotel_location["results"][0]["formatted_address"])
print(hotel_location["results"][0]["plus_code"]



#print(json.dumps(hotel_location, indent=4, sort_keys=True))

Arawa
Arawa, Papua New Guinea
{
    "html_attributions": [],
    "results": [
        {
            "formatted_address": "Arawa, Papua New Guinea",
            "geometry": {
                "location": {
                    "lat": -6.225624199999999,
                    "lng": 155.5678146
                },
                "viewport": {
                    "northeast": {
                        "lat": -6.2199824,
                        "lng": 155.5767166
                    },
                    "southwest": {
                        "lat": -6.236290700000001,
                        "lng": 155.5563738
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png",
            "name": "Arawa",
            "photos": [
                {
                    "height": 1600,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/1116201571954

In [43]:

print(hotel_location["results"][0]["name"])
print(hotel_location["results"][0]["formatted_address"])

Arawa
Arawa, Papua New Guinea


In [61]:
hotel_df


,Hotel Name,Hotel Address,Hotel Rating,Lat,Lng,Match Address
0,The Green Park Hotel Diyarbakır *****,"No:, Gazi Cd. No:21, Sur",4.6,37.910188,40.236627,
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209,
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832,


In [64]:


#lat = hotel_df.loc[city]["Lat"]
#lng = hotel_df.loc[city]["Lng"]
#lat = hotel_df["LAt"]
#lat = hotel_df["Lng"]


for hotel in hotel_df["Hotel Address"]:

    lat = hotel_df["Lat"]
    lng = hotel_df["Lng"]

    hotel_coords = f"{lat},{lng}"


    params_Q = {
            "location": hotel_coords,
            "types": hotel,
            "radius": 100,
            "key": g_key
    }


    # Text search
    text_base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

    text_base_2_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

    response = requests.get(text_base_url, params = params_Q)

    #print(response)


    hotel_location = response.json()

    #print(json.dumps(hotel_location, indent=4, sort_keys=True))

    #print(hotel_location["results"][0]["name"])

    print(hotel_location["results"][0]["name"])
    print(hotel_location["results"][0]["formatted_address"])
    #print(hotel_location["results"][0]["plus_code"]




IndexError: list index out of range

In [ ]:
#deliminate
df[['V','allele']] = df['V'].str.split(',',expand=True)

In [ ]:
tmpDF = pd.DataFrame(columns=['A','B'])
tmpDF[['A','B']] = df['V'].str.split('-', expand=True)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
mod_temp1_df = [info_box_template.format(**row) for index, row in mod_temp_df.iterrows()]
locations = mod_temp_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
